In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np 
import scipy.io as sio
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()
%matplotlib inline

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.set_default_dtype(torch.float32)

In [2]:
# Reading the Comp. Matrix - N x (D + y)
data = sio.loadmat("./Custom Graph Dataset/No_nan_matrix.mat")["training_matrix"]

# Feature Matrix - N x 2 (ax, ay)
X = data[:, :-1]

# Targets - N x 1 (vdot)
y = data[:, -1][:, np.newaxis]

# Normalization
mu = X.mean(axis=0)
sigma = X.std(axis=0)

# Normalized Dataset
X_norm = (X - mu) / sigma

X_norm.shape, y.shape, mu, sigma

((27435, 2),
 (27435, 1),
 array([-0.00133139, -0.01226881]),
 array([0.33017364, 0.18843928]))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((19753, 2), (2195, 2), (5487, 2), (19753, 1), (2195, 1), (5487, 1))

In [4]:
train_data = np.concatenate((X_train, y_train), axis=1)
val_data = np.concatenate((X_val, y_val), axis=1)
test_data = np.concatenate((X_test, y_test), axis=1)

train_data.shape, val_data.shape, test_data.shape

((19753, 3), (2195, 3), (5487, 3))

In [5]:
np.save("./Custom Graph Dataset/train_cg.npy", train_data.astype(np.float32))
np.save("./Custom Graph Dataset/val_cg.npy", val_data.astype(np.float32))
np.save("./Custom Graph Dataset/test_cg.npy", test_data.astype(np.float32))

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
# Expressive GRU
class ProGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ProGRU, self).__init__()
        self.lstm = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=2, bidirectional=True, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.PReLU()
    
    def forward(self, x):
        batch_size = x.shape[0]
        out, hn = self.lstm(x.view(batch_size, 1, -1))
        out = self.activation(self.linear1(hn[0])) # self.linear1(out)
        out = self.activation(self.linear2(out))
        out = self.linear3(out)
        return out

In [15]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
    """Expert Trajectory Dataset."""
    def __init__(self, data):
        
        # Inputs
        self.x = data[:, :-1]
        
        # Output
        self.y = data[:, -1]
        
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        
        # Inputs
        inp = self.x[idx]
        inp = inp.flatten()
        
        # Outputs
        out = self.y[idx]
        out = out.flatten()
        
        return torch.tensor(inp).float(), torch.tensor(out).float()

# Load the dataset
train_data = np.load("./Custom Graph Dataset/train_cg.npy")
val_data = np.load("./Custom Graph Dataset/val_cg.npy")
test_data = np.load("./Custom Graph Dataset//test_cg.npy")

# Using PyTorch Dataloader
train_dataset = TrajDataset(train_data)
val_dataset = TrajDataset(val_data)
test_dataset = TrajDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [16]:
for batch_num, (datas) in enumerate(train_loader):
    inp, out = datas
    print(inp.shape, out.shape)
    break

torch.Size([1, 2]) torch.Size([1, 1])


In [18]:
len(train_loader)

19753

In [ ]:
 # GRU Inputs
input_size = 2
hidden_size = 256
output_size = 1

# FNN
model = ProGRU(input_size, hidden_size, output_size).to(device)
optimizer = optim.AdamW(model.parameters(), lr = 1e-4)
loss_fn = nn.MSELoss()

# Initial Velocity
v = torch.zeros_like(len(train_loader), requires_grad=True).to(device)
delta_t = 0.05

epochs = 2
avg_train_loss = []
for epoch in range(epochs):
    # Train Loop
    losses_train = []
    model.train()
    for inp, out in train_loader:

        inp = inp.to(device)
        out = out.to(device)

        # Forward Pass
        pred = model(inp)
        loss = loss_fn(pred, out)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses_train.append(loss.detach().cpu().numpy())
        
    print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}")
    avg_train_loss.append(np.average(losses_train))

### Two Baselines

RNN Neural Network

x1 = [ax1 ay1] at t1

x1 -> h1 -> v1_hat (h0, v0) initial

x2 -> h2 + h1 + v1_hat -> v2_hat and so on 

MSELoss (v_hat, v_gt)

Approach 2: Map (ax, ay) -> v2 - v1 (Take Diff over the column)

Discard the last row from the training set :)